In [1]:
from bs4 import BeautifulSoup
import configparser
import requests
import pandas as pd
import pickle
from constants_funcs import get_session, int_or_float_or_str
import json

with open('constants.json', 'r') as d:
    constants = json.load(d)
    
sess = get_session()
sess




search_term = '' # if you want to find specific players by name change this search term
all_id_url = constants['LLHEADER'] + '/backend-search.php?term=' + search_term
all_id_url

ids_html = sess.get(all_id_url)
html_text = ids_html.text
soup = BeautifulSoup(html_text, 'html.parser')

rows = soup.find_all('a')
dic = dict() # CAVEAT: ONLY FINDS CURRENTLY ACTIVE PLAYERS
for row in rows:
    id = int(row['href'].split('?')[1])
    name = row.find('span').text
    loc = row.find('i').text
    dic[id] = {'name': name, 'loc': loc}

    
    
    
    
def get_children(id):
    referrals_url = constants['LLHEADER'] + f'/profiles.php?{str(id)}&5'
    print(referrals_url)
    main_data = sess.get(referrals_url)
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    header = soup.find(text="Direct Referrals ").next_element.next_element.next_element.next_element
    return header


def init_profile(id):
    url = constants['LLHEADER'] + f'/profiles.php?{str(id)}'
    main_data = sess.get(url)
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    ret = dict()

    if 'This player is inactive.' in soup.text:
        ret['status'] = 'inactive'
        ret['name'] = soup.find('div', attrs={'class': 'namediv'}).text.strip()
    else:
        box = soup.find('div', attrs={'class': 'topcont'})
        if '\t\tPassed away' in box.text:
            ret['status'] = 'deceased'
        else:
            ret['status'] = box['class'][-1]
        ret['branch'] = soup.findAll('div', attrs={'class': 'demog_row'})[1].text.strip().split()[1]
        genloc = soup.findAll('div', attrs={'class': 'demog_row'})[0].text.strip().split('\n\n')
        ret['gender'] = genloc[0].strip().split(': ')[-1]
        ret['location'] = genloc[1].strip().split(': ')[-1]
        ret['referrer'] = int(soup.findAll('div', attrs={'class': 'demog'})[4].find('a')['href'].split('?')[-1])
        ret['name'] = soup.find('h1', attrs={'class': 'namecss'}).text
        ret['league'] = soup.find('div', attrs={'class': 'leaguelogodiv'}).text.strip()
        record_rows = soup.find('table', attrs={'summary': 'Data table for career record'}).findAll('tr')[1:]
        record_dic = dict()
        for row in record_rows:
            level, record, _ = row.text.strip().split('\n')
            level = level.split(' ')[-1]
            record_dic[level] = {
                'W': int(record.split('-')[0]),
                'L': int(record.split('-')[1]),
                'T': int(record.split('-')[2]),
            }
        ret['record'] = pd.DataFrame.from_dict(record_dic, orient='index')
        data_table = soup.find('table', attrs={'class': 'std sortable this_sea std_bord'})
        rows = data_table.findAll('tr')
        headers = ['Category', 'Career', 'Career Pct', 'Lg Pct', 'Recent Szn', 'Szn Pct', 'Szn Lg Pct', 'Rundle Pct']
        tbl = [[int_or_float_or_str(x.text) for x in row.findAll('td')] for row in rows[1:]]
        df = pd.DataFrame(tbl, columns=headers)
        df['Career TCA'] = df.apply(lambda row: int(row['Career'].split('-')[0]), axis=1)
        df['Career Q'] = df.apply(lambda row: int(row['Career'].split('-')[1]), axis=1)
        df['Szn TCA'] = df.apply(lambda row: int(row['Recent Szn'].split('-')[0]), axis=1)
        df['Szn Q'] = df.apply(lambda row: int(row['Recent Szn'].split('-')[1]), axis=1)
        df.drop(['Career', 'Career Pct', 'Recent Szn', 'Szn Pct'], axis=1, inplace=True)
        new_headers = ['Category', 'Career TCA', 'Career Q', 'Lg Pct', 'Szn TCA', 'Szn Q', 'Szn Lg Pct', 'Rundle Pct']
        df = df[new_headers]
        ret['Q_data'] = df
    return ret


using existing login session


In [2]:
questions = pd.read_csv('./FULL_QUESTION_HISTORY.csv')

def build_prediction(profileno,seas,mday):
    myprofile = init_profile(profileno)
    tca = myprofile['Q_data']['Career TCA']
    q = myprofile['Q_data']['Career Q']
    pct = tca/q
    q_cat = myprofile['Q_data']['Category']
    cat_pct = dict()
    for x in range(len(q_cat)):
        cat_pct[q_cat[x]] = float(pct[x])
    
    md = questions[(questions.season == seas)&(questions.matchday == mday)].reset_index()
    ex = md.apply(lambda row: row['defense'] * cat_pct[row['category']], axis=1)
    xPTS = sum(ex)
    xCA = sum([cat_pct[cat] for cat in md['category']])
    
    daily_catpct = [cat_pct[cat] for cat in md['category']]
    
    md['cat_pct'] = daily_catpct
    
    md.sort_values(by=['cat_pct'], inplace=True)
    md['ideal_d'] = [3,2,2,1,1,0]
    
    ex = md.apply(lambda row: row['ideal_d'] * cat_pct[row['category']], axis=1)
    xPTS_ideal = sum(ex)
    
    md['worst_d'] = [0,1,1,2,2,3]
    
    ex = md.apply(lambda row: row['worst_d'] * cat_pct[row['category']], axis=1)
    xPTS_worst = sum(ex)
    
    return xCA, xPTS, xPTS_ideal, xPTS_worst

In [4]:
build_prediction(16642,92,1)

(2.9834430897939948, 4.403851190522072, 3.9532883259298357, 4.997040943452148)